In [1]:
#conexão com o Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#cria a pasta para manuseio do Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [0]:
def process_data(df_train, df_test, df_answer):

	features = list(df_test)
	features.append('NU_NOTA_MT')

	df_answer['NU_INSCRICAO'] = df_test['NU_INSCRICAO']

	df_train = df_train[features]

	# Dropando tabelas que não serão utilizadas

	df_train.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	df_test.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	# Processado dados categóricos

	df_train['TP_SEXO'].replace('M', 0, inplace=True)
	df_train['TP_SEXO'].replace('F', 1, inplace=True)

	df_test['TP_SEXO'].replace('M', 0, inplace=True)
	df_test['TP_SEXO'].replace('F', 1, inplace=True)

	# Coloquei as notas como zero pq acredito que representam a nota dos alunos que faltaram.

	df_train['NU_NOTA_CN'].fillna(0, inplace=True)
	df_train['NU_NOTA_CH'].fillna(0, inplace=True)
	df_train['NU_NOTA_LC'].fillna(0, inplace=True)
	df_train['NU_NOTA_MT'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_train['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	df_test['NU_NOTA_CN'].fillna(0, inplace=True)
	df_test['NU_NOTA_CH'].fillna(0, inplace=True)
	df_test['NU_NOTA_LC'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_test['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	# Coloquei como 8 pq acredito que represente uma situação neutra

	df_train['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	df_test['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	# One_hot_enconding em variáveis categóricas

	df_train = pd.get_dummies(df_train, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	df_test = pd.get_dummies(df_test, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	# Utilizando Estatística Descritiva

	notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
	
	df_train['KURTOSIS_NOTAS'] = df_train[notas].kurtosis(axis=1)
	df_train['MEAN_NOTAS'] = df_train[notas].mean(axis=1)
	df_train['MEDIAN_NOTAS'] = df_train[notas].median(axis=1)
	df_train['MAD_NOTAS'] = df_train[notas].mad(axis=1)
	df_train['QUANTILE_NOTAS'] = df_train[notas].quantile(axis=1)
	df_train['SEM_NOTAS'] = df_train[notas].sem(axis=1)
	df_train['SKEW_NOTAS'] = df_train[notas].skew(axis=1)
	df_train['STD_NOTAS'] = df_train[notas].std(axis=1)
	df_train['VAR_NOTAS'] = df_train[notas].var(axis=1)
	df_train['AMP_NOTAS'] = df_train[notas].max(axis=1) - df_train[notas].min(axis=1)
	
	df_train['MEAN_NOTA_CH_LC'] = df_train[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CH_CN'] = df_train[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_train['MEAN_NOTA_CH_REDACAO'] = df_train[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_CN_LC'] = df_train[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CN_REDACAO'] = df_train[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_LC_REDACAO'] = df_train[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)
	
	notas_red = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

	df_train['KURTOSIS_NOTAS_COMP'] = df_train[notas_red].kurtosis(axis=1)
	df_train['MEAN_NOTAS_COMP'] = df_train[notas_red].mean(axis=1)
	df_train['MEDIAN_NOTAS_COMP'] = df_train[notas_red].median(axis=1)
	df_train['MAD_NOTAS_COMP'] = df_train[notas_red].mad(axis=1)
	df_train['QUANTILE_NOTAS_COMP'] = df_train[notas_red].quantile(axis=1)
	df_train['SEM_NOTAS_COMP'] = df_train[notas_red].sem(axis=1)
	df_train['STD_NOTAS_COMP'] = df_train[notas_red].std(axis=1)
	df_train['VAR_NOTAS_COMP'] = df_train[notas_red].var(axis=1)
	df_train['AMP_NOTAS_COMP'] = df_train[notas_red].max(axis=1) - df_train[notas_red].min(axis=1)

	df_train['MEAN_NOTA_COMP1_COMP2'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP3'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP4'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP5'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP3'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP4'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP5'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP4'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP5'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP4_COMP5'] = df_train[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)
	
	df_test['KURTOSIS_NOTAS'] = df_test[notas].kurtosis(axis=1)
	df_test['MEAN_NOTAS'] = df_test[notas].mean(axis=1)
	df_test['MEDIAN_NOTAS'] = df_test[notas].median(axis=1)
	df_test['MAD_NOTAS'] = df_test[notas].mad(axis=1)
	df_test['QUANTILE_NOTAS'] = df_test[notas].quantile(axis=1)
	df_test['SEM_NOTAS'] = df_test[notas].sem(axis=1)
	df_test['SKEW_NOTAS'] = df_test[notas].skew(axis=1)
	df_test['STD_NOTAS'] = df_test[notas].std(axis=1)
	df_test['VAR_NOTAS'] = df_test[notas].var(axis=1)
	df_test['AMP_NOTAS'] = df_test[notas].max(axis=1) - df_test[notas].min(axis=1)
	
	df_test['MEAN_NOTA_CH_LC'] = df_test[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CH_CN'] = df_test[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_test['MEAN_NOTA_CH_REDACAO'] = df_test[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_CN_LC'] = df_test[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CN_REDACAO'] = df_test[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_LC_REDACAO'] = df_test[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)
	
	df_test['KURTOSIS_NOTAS_COMP'] = df_test[notas_red].kurtosis(axis=1)
	df_test['MEAN_NOTAS_COMP'] = df_test[notas_red].mean(axis=1)
	df_test['MEDIAN_NOTAS_COMP'] = df_test[notas_red].median(axis=1)
	df_test['MAD_NOTAS_COMP'] = df_test[notas_red].mad(axis=1)
	df_test['QUANTILE_NOTAS_COMP'] = df_test[notas_red].quantile(axis=1)
	df_test['SEM_NOTAS_COMP'] = df_test[notas_red].sem(axis=1)
	df_test['STD_NOTAS_COMP'] = df_test[notas_red].std(axis=1)
	df_test['VAR_NOTAS_COMP'] = df_test[notas_red].var(axis=1)
	df_test['AMP_NOTAS_COMP'] = df_test[notas_red].max(axis=1) - df_test[notas_red].min(axis=1)

	df_test['MEAN_NOTA_COMP1_COMP2'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP3'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP4'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP5'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP3'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP4'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP5'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP4'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP5'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP4_COMP5'] = df_test[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)

	return df_train, df_test, df_answer

In [0]:
df_train = pd.read_csv('/content/drive/ML/data-codenation-challenge/train.csv')
df_test = pd.read_csv('/content/drive/ML/data-codenation-challenge/test.csv')
df_answer = pd.DataFrame()

df_train, df_test, df_answer = process_data(df_train, df_test, df_answer)

In [0]:
label = df_train['NU_NOTA_MT']

df_train.drop(['NU_NOTA_MT'], axis=1, inplace=True)

In [10]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = df_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               14592     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 179,457
Trainable params: 179,457
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = '/content/drive/ML/data-codenation-challenge/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [11]:
NN_model.fit(df_train, label, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 10984 samples, validate on 2746 samples
Epoch 1/500
10984/10984 [==============================] - 4s 377us/step - loss: 65.1988 - mean_absolute_error: 65.1988 - val_loss: 55.6846 - val_mean_absolute_error: 55.6846

Epoch 00001: val_loss improved from inf to 55.68461, saving model to /content/drive/ML/data-codenation-challenge/Weights-001--55.68461.hdf5
Epoch 2/500
10984/10984 [==============================] - 2s 147us/step - loss: 53.8571 - mean_absolute_error: 53.8571 - val_loss: 51.2488 - val_mean_absolute_error: 51.2488

Epoch 00002: val_loss improved from 55.68461 to 51.24883, saving model to /content/drive/ML/data-codenation-challenge/Weights-002--51.24883.hdf5
Epoch 3/500
10984/10984 [==============================] - 2s 144us/step - loss: 49.6357 - mean_absolute_error: 49.6357 - val_loss: 46.8099 - val_mean_absolute_error: 46.8099

Epoch 00003: val_loss improved from 51.24883 to 46.80986, saving model to /content/drive/M

In [0]:
# Load wights file of the best model :
wights_file = '/content/drive/ML/data-codenation-challenge/Weights-479--42.68452.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
predictions = NN_model.predict(df_test)

In [0]:
df_answer['NU_NOTA_MT'] = np.around(predictions,2)

df_answer.to_csv('/content/drive/ML/data-codenation-challenge/dnn_answer.csv', index=False, header=True)

In [0]:
def convert_to_zero(filename):

	data = pd.read_csv(filename+'.csv')

	for i in range(0, data.shape[0]):
		if data['NU_NOTA_MT'][i] < 80:
			data['NU_NOTA_MT'][i] = 0.00

	data.to_csv('/content/drive/ML/data-codenation-challenge/xgb_dnn.csv', index=False, header=True)

	return data

In [18]:
XGBModel = XGBRegressor()
XGBModel.fit(df_train,label, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [0]:
XGBpredictions = XGBModel.predict(df_test)

In [0]:
df_answer['NU_NOTA_MT'] = np.around(XGBpredictions,2)

df_answer.to_csv('/content/drive/ML/data-codenation-challenge/xgb_answer.csv', index=False, header=True)

data = convert_to_zero('/content/drive/ML/data-codenation-challenge/xgb_answer')